In [3]:
import numpy as np
import pandas as pd
import pickle

In [40]:
input_id = 0
st = 'intersection'

PATH = "Classification/Classifiers_result__{}__%i__{}__.pkl".format(input_id, st)


N = 5

In [41]:
df = pd.DataFrame()

for i in range(N):
    results_dict = pickle.load(open(PATH %(i),'rb'))

    for gra in results_dict.keys():
        if type(results_dict[gra]) != str:
            n_classifiers = len(results_dict[gra].keys())
            for clf in results_dict[gra].keys():
                aux_dict = results_dict[gra][clf]
                aux_dict['Granularity'] = gra
                aux_dict['Classifier'] = clf
                aux_dict['Iteration'] = i
                aux_dict['Time'] = results_dict[gra][clf]['time']
                df = df.append(aux_dict, ignore_index=True)

In [42]:
df['Time[min]'] = df['Time'].dt.total_seconds()/(100*60)


In [43]:
results_df = df.groupby(['Classifier','Granularity']).agg({'Accuracy':['mean','std']})*100

In [38]:
results_df

Accuracy          
                                mean       std
Classifier    Granularity                     
AdaBoost      2.0          46.488019  3.552184
              3.0          46.488019  3.552184
              4.0          47.436349  3.862959
              5.0          48.612025  2.512317
              6.0          45.810461  5.684985
...                              ...       ...
Random Forest 6.0          48.612571  2.600706
              7.0          47.779237  3.569515
              8.0          50.143737  6.095058
              9.0          52.112212  9.676872
              10.0         49.437969  5.011294

[90 rows x 2 columns]

In [39]:
results_df.loc[(slice(None),3), :]


Accuracy          
                                    mean       std
Classifier        Granularity                     
AdaBoost          3.0          46.488019  3.552184
Decision Tree     3.0          46.488019  3.552184
Gaussian Process  3.0          46.488019  3.552184
Linear SVM        3.0          46.548079  3.461504
Naive Bayes       3.0          46.548079  3.461504
Nearest Neighbors 3.0          46.548079  3.461504
Neural Net        3.0          47.321352  4.987298
QDA               3.0          46.548079  3.461504
RBF SVM           3.0          45.349960  3.565485
Random Forest     3.0          46.488019  3.552184